# Skimming large amounts of Sentinel 2 data
In this notebook, we will show how using `geolt.skim` and `geolt.summarize` can help prepare and visualize large sets of data. We will load a large set of Sentinel-2 imagery over Austin, TX and quickly retreive statistics about the features within to assist with downstream processing. We will use the following process:
1. Initialize Dask cluster to load `xarray Dataset` from a public S3 bucket.
2. `skim` the dataset for a quick set of summary statistics.
3. `summarize` features within the dataset to assist with feature engineering and downstream use.

In [20]:
from porcupine import skim, visualize, query
import pandas as pd
import xarray
import dask.distributed
from odc.stac import configure_rio, stac_load
from pystac_client import Client
import folium
import folium.plugins
import geopandas as gpd
import shapely.geometry
from IPython.display import HTML, display

client = dask.distributed.Client()
configure_rio(cloud_defaults=True, aws={"aws_unsigned": True}, client=client)
display(client)

/Users/dylan/opt/anaconda3/envs/odc-stac/lib/python3.10/site-packages/distributed/node.py:179: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 65431 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:65431/status,
Dashboard: http://127.0.0.1:65431/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:65432,Workers: 4
Dashboard: http://127.0.0.1:65431/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:65450,Total threads: 2
Dashboard: http://127.0.0.1:65451/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:65437,


## Load Dataset from STAC catalog
The following cell loads an `xarray Dataset` containing Sentinel-2 imagery from a STAC catalog. We initialize the center point of our query to be the center of Austin, TX and search for imagery from July, 2022 within a 100 km radius. 

In [26]:
config = {
    "sentinel-s2-l2a-cogs": {
        "assets": {
            "*": {"data_type": "uint16", "nodata": 0},
            "SCL": {"data_type": "uint8", "nodata": 0},
            "visual": {"data_type": "uint8", "nodata": 0},
        },
        "aliases": {"red": "B04", "green": "B03", "blue": "B02"},
    },
    "*": {"warnings": "ignore"},
}
km2deg = 1.0 / 111
x, y = (-97.744, 30.266)
r = 100 * km2deg
bbox = (x - r, y - r, x + r, y + r)
catalog = Client.open("https://earth-search.aws.element84.com/v0")
query = catalog.search(
    collections=["sentinel-s2-l2a-cogs"], datetime="2022-07-01/2022-07-31", limit=100, bbox=bbox
)
items = list(query.get_items())
stac_json = query.get_all_items_as_dict()

crs = "epsg:3857"
zoom = 2**5 # overview level 5

data = stac_load(
    items,
    crs=crs,
    resolution=10*zoom,
    chunks={},
    groupby="solar_day",
    stac_cfg=config,
)
data

<xarray.Dataset>
Dimensions:      (y: 1142, x: 1137, time: 2)
Coordinates:
  * y            (y) float64 3.716e+06 3.715e+06 ... 3.351e+06 3.351e+06
  * x            (x) float64 -1.102e+07 -1.102e+07 ... -1.066e+07 -1.066e+07
    spatial_ref  int32 3857
  * time         (time) datetime64[ns] 2022-07-28T17:25:14 2022-07-30T17:15:20
Data variables: (12/16)
    visual       (time, y, x) uint8 dask.array<chunksize=(1, 1142, 1137), meta=np.ndarray>
    B01          (time, y, x) uint16 dask.array<chunksize=(1, 1142, 1137), meta=np.ndarray>
    B02          (time, y, x) uint16 dask.array<chunksize=(1, 1142, 1137), meta=np.ndarray>
    B03          (time, y, x) uint16 dask.array<chunksize=(1, 1142, 1137), meta=np.ndarray>
    B04          (time, y, x) uint16 dask.array<chunksize=(1, 1142, 1137), meta=np.ndarray>
    B05          (time, y, x) uint16 dask.array<chunksize=(1, 1142, 1137), meta=np.ndarray>
    ...           ...
    B09          (time, y, x) uint16 dask.array<chunksize=(1, 1142, 1137), meta=np.ndarray>
    B11          (time, y, x) uint16 dask.array<chunksize=(1, 1142, 1137), meta=np.ndarray>
    B12          (time, y, x) uint16 dask.array<chunksize=(1, 1142, 1137), meta=np.ndarray>
    AOT          (time, y, x) uint16 dask.array<chunksize=(1, 1142, 1137), meta=np.ndarray>
    WVP          (time, y, x) uint16 dask.array<chunksize=(1, 1142, 1137), meta=np.ndarray>
    SCL          (time, y, x) uint8 dask.array<chunksize=(1, 1142, 1137), meta=np.ndarray>

In [27]:
stac_json

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'stac_version': '1.0.0-beta.2',
   'stac_extensions': ['eo', 'view', 'proj'],
   'id': 'S2A_14RNT_20220730_0_L2A',
   'bbox': [-98.24017654176154,
    28.834781500810376,
    -97.86379644637883,
    29.82659734364876],
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-97.87472854105172, 28.834781500810376],
      [-98.24017654176154, 28.837334879225125],
      [-97.99000597196367, 29.82659734364876],
      [-97.86379644637883, 29.82557210301281],
      [-97.87472854105172, 28.834781500810376]]]},
   'properties': {'datetime': '2022-07-30T17:15:56Z',
    'platform': 'sentinel-2a',
    'constellation': 'sentinel-2',
    'instruments': ['msi'],
    'gsd': 10,
    'view:off_nadir': 0,
    'proj:epsg': 32614,
    'sentinel:utm_zone': 14,
    'sentinel:latitude_band': 'R',
    'sentinel:grid_square': 'NT',
    'sentinel:sequence': '0',
    'sentinel:product_id': 'S2A_MSIL2A_20220730T165901_N0400_R069_T14RNT_20220731T1

## Perform a local skim of the data for 1 time slice

In [3]:
small_data = data.sel(time='2022-07-03').compute()

In [4]:
%%time
df = skim(small_data)
df

CPU times: user 164 ms, sys: 45.9 ms, total: 209 ms
Wall time: 208 ms


,variables,data_types,NaNs,mean,std,maximums,minimums
0,visual,uint8,False,83.206896,85.090139,255,0
1,B01,uint16,False,694.111545,1057.890177,12671,0
2,B02,uint16,False,715.116638,1019.690505,11615,0
3,B03,uint16,False,831.311709,1033.465046,10787,0
4,B04,uint16,False,897.211163,1064.686861,10284,0
5,B05,uint16,False,1144.481014,1258.273113,11399,0
6,B06,uint16,False,1453.429606,1447.058901,10691,0
7,B07,uint16,False,1585.919234,1543.215470,10361,0
8,B08,uint16,False,1592.048660,1549.807719,10519,0
9,B8A,uint16,False,1737.786114,1662.727114,10352,0


In [5]:
gdf = gpd.GeoDataFrame.from_features(stac_json,3857)
gdf

,geometry,datetime,platform,constellation,instruments,gsd,view:off_nadir,proj:epsg,sentinel:utm_zone,sentinel:latitude_band,sentinel:grid_square,sentinel:sequence,sentinel:product_id,sentinel:data_coverage,eo:cloud_cover,sentinel:valid_cloud_cover,sentinel:processing_baseline,sentinel:boa_offset_applied,created,updated
0,"POLYGON ((-97.875 28.835, -98.240 28.837, -97....",2022-07-30T17:15:56Z,sentinel-2a,sentinel-2,[msi],10,0,32614,14,R,NT,0,S2A_MSIL2A_20220730T165901_N0400_R069_T14RNT_2...,21.79,0.00,True,04.00,True,2022-07-31T19:25:18.048Z,2022-07-31T19:25:18.048Z
1,"POLYGON ((-96.850 28.822, -97.975 28.836, -97....",2022-07-30T17:15:52Z,sentinel-2a,sentinel-2,[msi],10,0,32614,14,R,PT,0,S2A_MSIL2A_20220730T165901_N0400_R069_T14RPT_2...,100.00,8.71,True,04.00,True,2022-07-31T20:23:18.094Z,2022-07-31T20:23:18.094Z
2,"POLYGON ((-95.827 28.802, -96.951 28.824, -96....",2022-07-30T17:15:49Z,sentinel-2a,sentinel-2,[msi],10,0,32614,14,R,QT,0,S2A_MSIL2A_20220730T165901_N0400_R069_T14RQT_2...,100.00,85.35,True,04.00,True,2022-07-31T19:51:10.772Z,2022-07-31T19:51:10.772Z
3,"POLYGON ((-97.866 29.737, -98.012 29.739, -97....",2022-07-30T17:15:44Z,sentinel-2a,sentinel-2,[msi],10,0,32614,14,R,NU,0,S2A_MSIL2A_20220730T165901_N0400_R069_T14RNU_2...,3.91,5.61,True,04.00,True,2022-07-31T20:11:50.723Z,2022-07-31T20:11:50.723Z
4,"POLYGON ((-96.831 29.724, -97.966 29.738, -97....",2022-07-30T17:15:37Z,sentinel-2a,sentinel-2,[msi],10,0,32614,14,R,PU,0,S2A_MSIL2A_20220730T165901_N0400_R069_T14RPU_2...,93.10,2.94,True,04.00,True,2022-07-31T19:56:23.705Z,2022-07-31T19:56:23.705Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,"POLYGON ((-97.686 28.835, -97.975 28.836, -97....",2022-07-03T17:25:51Z,sentinel-2a,sentinel-2,[msi],10,0,32614,14,R,PT,0,S2A_MSIL2A_20220703T170901_N0400_R112_T14RPT_2...,37.38,69.16,True,04.00,True,2022-07-04T03:04:36.967Z,2022-07-04T03:04:36.967Z
80,"POLYGON ((-97.865 29.737, -99.000 29.742, -99....",2022-07-03T17:25:40Z,sentinel-2a,sentinel-2,[msi],10,0,32614,14,R,NU,0,S2A_MSIL2A_20220703T170901_N0400_R112_T14RNU_2...,100.00,47.54,True,04.00,True,2022-07-04T03:11:37.286Z,2022-07-04T03:11:37.286Z
81,"POLYGON ((-97.431 29.734, -97.966 29.739, -97....",2022-07-03T17:25:36Z,sentinel-2a,sentinel-2,[msi],10,0,32614,14,R,PU,0,S2A_MSIL2A_20220703T170901_N0400_R112_T14RPU_2...,58.78,16.39,True,04.00,True,2022-07-04T03:08:20.582Z,2022-07-04T03:08:20.582Z
82,"POLYGON ((-97.854 30.640, -99.000 30.645, -99....",2022-07-03T17:25:26Z,sentinel-2a,sentinel-2,[msi],10,0,32614,14,R,NV,0,S2A_MSIL2A_20220703T170901_N0400_R112_T14RNV_2...,100.00,33.68,True,04.00,True,2022-07-04T03:08:16.110Z,2022-07-04T03:08:16.110Z


In [6]:
bbox2 = (gdf.bounds['minx'].min(),gdf.bounds['miny'].min(),gdf.bounds['maxx'].max(),gdf.bounds['maxy'].max())
bbox2

(-99.00020036474238, 28.8021464220105, -95.7356923552778, 31.63553809078258)

In [7]:
bbox3 = tuple([b+2 for b in bbox2])
bbox3

(-97.00020036474238, 30.8021464220105, -93.7356923552778, 33.635538090782575)

In [21]:
query(bbox=[bbox,bbox2,bbox3],
                    name=["Query","Returned","Test"],
                    m=folium.Map(),
                    color=["red","green","blue"])

In [ ]:
B = [bbox,bbox2,bbox3]
N = ["Query","Returned","Test"]
C = ["red","darkblue","cadetblue"]
for (b,n,c) in zip(B,N,C):
    print(f'{b}->{n}->{c}')

In [ ]:
def convert_bounds(bbox, invert_y=False):
    """
    Helper method for changing bounding box representation to leaflet notation

    ``(lon1, lat1, lon2, lat2) -> ((lat1, lon1), (lat2, lon2))``
    """
    x1, y1, x2, y2 = bbox
    if invert_y:
        y1, y2 = y2, y1
    return ((y1, x1), (y2, x2))

def compute_center(bbox):
    x1, y1, x2, y2 = bbox
    mu_x = (x1+x2)/2
    mu_y = (y1+y2)/2
    return (mu_y,mu_x)

map1 = folium.Map()

folium.GeoJson(
    shapely.geometry.box(*bbox),
    style_function=lambda x: dict(fill=False, 
                                  weight=5, 
                                  opacity=0.5, 
                                  color="green"),
    name="Query",
    tooltip="Query",
).add_to(map1)

folium.GeoJson(
    shapely.geometry.box(*bbox2),
    style_function=lambda x: dict(fill=False,
                                  weight=5, 
                                  opacity=0.5, 
                                  color="blue"),
    name="Returned",
    tooltip="Returned",
).add_to(map1)

folium.Marker(compute_center(bbox),
              popup="Query Center",
              icon=folium.Icon(color="green",icon="star")
             ).add_to(map1)
folium.Marker(compute_center(bbox2),
              popup="Return Center",
              icon=folium.Icon(color="blue", icon="star")
             ).add_to(map1)

map1.fit_bounds(bounds=convert_bounds(bbox))

map1

In [ ]:
# Compute granule id from components
gdf["granule"] = (
    gdf["sentinel:utm_zone"].apply(lambda x: f"{x:02d}")
    + gdf["sentinel:latitude_band"]
    + gdf["sentinel:grid_square"]
)

fig = gdf.plot(
    "granule",
    edgecolor="black",
    categorical=True,
    aspect="equal",
    alpha=0.5,
    figsize=(6, 12),
    legend=True,
    legend_kwds={"loc": "upper left", "frameon": False, "ncol": 1},
)
_ = fig.set_title("STAC Query Results")

In [ ]:
# from branca.element import Figure
def convert_bounds(bbox, invert_y=False):
    """
    Helper method for changing bounding box representation to leaflet notation

    ``(lon1, lat1, lon2, lat2) -> ((lat1, lon1), (lat2, lon2))``
    """
    x1, y1, x2, y2 = bbox
    if invert_y:
        y1, y2 = y2, y1
    return ((y1, x1), (y2, x2))

map1 = folium.Map()

folium.GeoJson(
    shapely.geometry.box(*bbox),
    style_function=lambda x: dict(fill=True, weight=1, opacity=0.5, color="green"),
    name="Query",
).add_to(map1)

# folium.GeoJson(
#     shapely.geometry.box(*bbox2),
#     style_function=lambda x: dict(fill=True, weight=1, opacity=0.5, color="blue"),
#     name="Returned",
# ).add_to(map1)
test_bounds = tuple([round(b) for b in bbox])
map1.fit_bounds(bounds=convert_bounds(test_bounds))
# display(fig)
map1

2